In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/')

In [2]:
#importing all the required packages
import optuna
from optuna.samplers import GridSampler, RandomSampler, TPESampler

import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools
import os
from pathlib import Path

from model.GRU_model import *
from pipeline_config import *
from utils import *

In [3]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs : {Epochs}')

Features: 1
HiddenSize: 64
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.1
Epochs : 30


In [4]:
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [5]:
def objective(trial):
    with open(train_dataset_path, 'rb') as f:
        train_dataset_dict = pickle.load(f)
        
    with open(test_dataset_path, 'rb') as f:
        test_dataset_dict = pickle.load(f)

    with open(category_id_to_category_name_path, 'rb') as f:
        category_id_to_name_dict = pickle.load(f)
        
    with open(categories_per_indent_path, 'rb') as f:
        categories_per_indent_dict = pickle.load(f)

    with open(son_parent_path, 'rb') as f:
        son_parent_dict = pickle.load(f)

    with open(parent_to_son_list_path, 'rb') as f:
        parent_to_son_list_dict = pickle.load(f)

    with open(hgru_model_weights_path, 'rb') as f:
        hgru_weight_dict = pickle.load(f)

    with open(coefficient_dict_path, 'rb') as f:
        coefficient_dict = pickle.load(f)
    
    weights_path = weightspath
    
    #--------------------------------------------------------------------------------------------------------------------------------------#
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights/'

    is_empty = not any(Path(desired_path).iterdir())
    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)

    #--------------------------------------------------------------------------------------------------------------------------------------#

    loss_coef_1 = trial.suggest_float('loss_coef_1',  1e-10, 1e-4, log=True)
    loss_coef_2 = trial.suggest_float('loss_coef_2',  1e-10, 1e-4, log=True)
    loss_coef_3 = trial.suggest_float('loss_coef_3',  1e-10, 1e-4, log=True)
    alpha = trial.suggest_float('alpha',  1e-10, 1, log=True)
    Lr = trial.suggest_float('Lr', 1e-5, 1e-1, log=True)

    bi_directional_models = {}
    num_categories = 0

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
 
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            min_error = training_and_evaluation(trial, model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, alpha, saving_param_path)
            
            bi_directional_models[category] = min_error

    average_error = sum(list(bi_directional_models.values()))/len(list(bi_directional_models.values()))
    
    return average_error


In [6]:
EPOCHS = 30
    
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.GPSampler())
study.optimize(objective, n_trials=50)

AttributeError: module 'optuna.samplers' has no attribute 'GPSampler'

In [ ]:
# Get the best trial
best_trial = study.best_trial

# Print the best trial
print('Best trial:')
print(f'Value: {best_trial.value}')
print('Parameters:')
for key, value in best_trial.params.items():
    print(f'{key}: {value}')

In [ ]:
EPOCHS = 50
    
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.RandomSampler())
study.optimize(objective, n_trials=50)

In [ ]:
# Get the best trial
best_trial = study.best_trial

# Print the best trial
print('Best trial:')
print(f'Value: {best_trial.value}')
print('Parameters:')
for key, value in best_trial.params.items():
    print(f'{key}: {value}')

In [ ]:
EPOCHS = 50
    
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=50)

In [ ]:
# Get the best trial
best_trial = study.best_trial

# Print the best trial
print('Best trial:')
print(f'Value: {best_trial.value}')
print('Parameters:')
for key, value in best_trial.params.items():
    print(f'{key}: {value}')